In [1]:
import pandas as pd
import json
from tabulate import tabulate
import pyperclip
import ast
from tqdm import tqdm

from libs import iou_evaluation

In [2]:
notation_df = pd.read_csv('api/common_core_data/Knowledge_Base/Coherence_map_Hai.csv')
notation_df.head()

,Grade,Subject Code,Subject,Subtopic,Description,Subtopic2,Description2,Subtopic12,Full Code,Sub Code
0,1,OA,Operations and Algebraic Thinking,A.1,Use addition and subtraction within 20 to solv...,NaN,NaN,A.1,1.OA.A.1,1.OA.1
1,1,OA,Operations and Algebraic Thinking,A.2,Solve word problems that call for addition of ...,NaN,NaN,A.2,1.OA.A.2,1.OA.2
2,1,NBT,Number and Operations in Base Ten,A.1,"Count to 120, starting at any number less than...",NaN,NaN,A.1,1.NBT.A.1,1.NBT.1
3,1,NBT,Number and Operations in Base Ten,B.2,Understand that the two digits of a two-digit ...,B.2.a,Understand that the two digits of a two-digit ...,B.2.a,1.NBT.B.2.a,1.NBT.2.a
4,1,NBT,Number and Operations in Base Ten,B.2,Understand that the two digits of a two-digit ...,B.2.b,The numbers from 11 to 19 are composed of a te...,B.2.b,1.NBT.B.2.b,1.NBT.2.b


In [3]:
model_list  = ['gemma-7b-it', 'mixtral-8x7b-32768', 'llama3-70b-8192', 'llama3-8b-8192', 'gpt-3.5-turbo']
model = model_list[0]
data_file = 'Grade_data_Train'
data_file+= '_from_Grade'

data = [
    ["Model", "Total IOU", "Score"]
]

kc_label = pd.read_csv(f'api/common_core_data/Test_Result/{data_file}/{data_file}_question_level_{model}.csv')
KC_question = {}

In [4]:
for i in tqdm(range(len(kc_label))):
    
    ques = kc_label.loc[i, 'Question']
    
    kcs = kc_label.loc[i, 'KCs applied']
    
    try:
        kcs = ast.literal_eval(kcs)
    except:
        kcs = [kcs]
        
    grade_applied = kc_label.loc[i, 'Grade applied']
    
    grade = kc_label.loc[i, 'Grade']
    
    kcs_pred = ast.literal_eval(kc_label.loc[i, 'KCs predicted'])
    
    KC_question[i] = {'Question': ques, 'KCs applied': kcs, 'Grade applied': grade_applied, 'Grade': grade, 'KCs predicted': kcs_pred}
    

100%|██████████| 613/613 [00:00<00:00, 18157.03it/s]


In [5]:
kc_df = pd.DataFrame(KC_question)
kc_df = kc_df.T
kc_df = kc_df.reset_index(drop=True)
kc_df.head(5)

,Question,KCs applied,Grade applied,Grade,KCs predicted
0,The graph below shows the numbers of laps four...,[3.MD.3],3,3,"[3.OA.A.1, 3.OA.A.3, 3.OA.C.7]"
1,Ms. Garcia wrote the sentence shown in the box...,[3.OA.1],3,3,"[3.OA.A.1, 3.OA.A.3, 3.OA.C.7]"
2,Ms. Shaw has a quilt that is in the shape of a...,[3.MD.8],3,3,"[3.OA.A.1, 3.OA.A.3, 3.NBT.A.2]"
3,Which of these sentences matches the expressio...,[3.OA.2],3,3,"[3.OA.A.2, 3.OA.A.3, 3.OA.D.8]"
4,What is the missing number that makes the numb...,[3.NBT.2],3,3,"[3.NBT.A.2, 3.OA.A.4, 3.OA.C.7]"


In [6]:
# Change the column name from sub_code to full_code
for i in range(0, kc_df.shape[0]):
    for j in range(0, len(kc_df.iloc[i,:]['KCs applied'])):
        kc = kc_df.iloc[i,:]['KCs applied'][j]
        if kc in notation_df['Sub Code'].values:
            kc_df.iloc[i,:]['KCs applied'][j] = notation_df[notation_df['Sub Code'] == kc ]['Full Code'].values[0]
    
    # grade_max = ''
    # grade_output = ''
    # for j in range(0, len(kc_df.iloc[i,:]['Grade applied'])):
    #     grade = kc_df.iloc[i,:]['Grade applied'][j]
    #     if grade == 'HS':
    #         grade_output +=grade + ','
    #     elif grade == 'K':
    #         continue
        
    #     else:
    #         if grade > grade_max:
    #             grade_max = grade
    # grade_output = grade_max
    
    # kc_df.iloc[i,:]['Grade applied'] = grade_output
            
        

In [7]:
kc_df['IOU'] = None
# Calculate IOU for each question
for i in range(kc_df.shape[0]):
    kc1 = kc_df.iloc[i]['KCs applied']
    kc2 = kc_df.iloc[i]['KCs predicted']
    
    # Ensure kc1 and kc2 are lists or sets
    kc1 = set(kc1) if isinstance(kc1, (list, set)) else set([kc1])
    kc2 = set(kc2) if isinstance(kc2, (list, set)) else set([kc2])
    
    intersection = len(kc1.intersection(kc2))
    union = len(kc1.union(kc2))
    iou = intersection / union if union != 0 else 0
    iou_formatted = format(iou, '.2f')
    
    kc_df.loc[i, 'IOU'] = float(iou_formatted)
    kc_df['IOU'] = pd.to_numeric(kc_df['IOU'], errors='coerce')

In [8]:
kc_df_sorted = kc_df.sort_values(by='IOU', ascending=False).reset_index(drop=True)
iou_total_sum = kc_df_sorted['IOU'].sum()
result = f'Total IOU over of model **{model}**: {iou_total_sum}/{len(kc_df_sorted)}'
pyperclip.copy(result)
print(result)

Total IOU over of model **gemma-7b-it**: 91.03/613


In [9]:
# List of all the models

model_list  = ['gemma-7b-it', 'mixtral-8x7b-32768', 'llama3-70b-8192', 'llama3-8b-8192', 'gpt-3.5-turbo']
model = model_list[0]
data_file = 'Grade_data_Train'
data_file+= '_from_Grade'

data = [
    ["Model", "Total IOU", "Score"]
]

for model in model_list:
    kc_label = pd.read_csv(f'api/common_core_data/Test_Result/{data_file}/{data_file}_question_level_{model}.csv')
    iou_total_sum, question_count = iou_evaluation(kc_label, notation_df)
    result = f'Total IOU over of model **{model}**: {iou_total_sum}/{question_count}'
    score = iou_total_sum/ question_count
    data.append([model, f'{iou_total_sum}/{question_count}', score])
    

100%|██████████| 613/613 [00:00<00:00, 37619.55it/s]


In [10]:
from tabulate import tabulate

for i in range(1, len(data)):
    data[i][2] = format(data[i][2], '.2f')
table = tabulate(data, headers="firstrow", tablefmt="pipe")
pyperclip.copy(table)

In [13]:
# kc_df_sorted = kc_df.sort_values(by='IOU', ascending=False).reset_index(drop=True)
# iou_total_sum = kc_df_sorted['IOU'].sum()


# full = f'**Total IOU for {kc_df.shape[0]} questions:** {iou_total_sum}\n\n'

# # Iterate over unique sources in kc_df_sorted
# for source in kc_df_sorted['Source'].unique():
#     # Filter DataFrame for the current source
#     kc_source = kc_df_sorted[kc_df_sorted['Source'] == source]
#     kc_source = kc_source.reset_index(drop=True)
    
#     table_str = tabulate(kc_source[['Grade applied','Grade predicted','IOU']], headers='keys', tablefmt='pipe')

#     full += f"**Ranked Questions based on IOU for {source}:**\n\n{table_str}\n\n"

# # Copy full formatted string to clipboard
# pyperclip.copy(full)

# # Notify user
# print("Formatted table copied to clipboard with bold headers.")